In [11]:
import scCube
from scCube import scCube
from scCube.visualization import *
from scCube.utils import *
import pandas as pd
import numpy as np
import warnings
import os
warnings.filterwarnings("ignore")
script_dir = os.path.dirname(os.path.abspath(os.getcwd()))

In [12]:
model = scCube()

In [13]:
sc_data = pd.read_csv(f'{script_dir}/Panel_D_src/Xenium_reference_count.csv', index_col=0)
sc_meta = pd.read_csv(f'{script_dir}/Panel_D_src/Xenium_reference_metadata.csv', index_col=0)
sc_meta.index = sc_meta['cell_id'].astype(str)
shared_index = sc_data.columns.intersection(sc_meta.index)
sc_data = sc_data[shared_index]
sc_meta = sc_meta.loc[shared_index]
sc_meta['Cell'] = sc_meta.index

In [14]:
sc_adata = model.pre_process(sc_data=sc_data, 
                             sc_meta=sc_meta,
                             is_normalized=False)

the input is count matrix, normalizing it firstly...


In [16]:
generate_sc_meta, generate_sc_data = model.train_vae_and_generate_cell(
    sc_adata=sc_adata,
    celltype_key='Cluster',
    cell_key='Cell',
    target_num=dict(sc_meta.Cluster.value_counts()), # target number of cells to generate, if `target_num=None`, generate cells by the proportion of cell types of the input data
    batch_size=512,
    epoch_num=1000,
    lr=0.0001,
    hidden_size=128,
    save_model=False,
    used_device='cpu',)

generating by the targeted proportion of cell types...
begin vae training...


Train Epoch: 999: 100%|██████████| 1000/1000 [02:13<00:00,  7.49it/s, loss=4.5321, min_loss=4.5321]


vae training done!


Generate Epoch: 0: 100%|██████████| 2032/2032 [00:00<00:00, 22446.80it/s]

generated done!
data have been prepared!


In [ ]:
for i in range(9):
    np.random.seed(i)
    _, generate_sc_meta_new = model.generate_pattern_random(
        generate_sc_data=generate_sc_data,
        generate_sc_meta=generate_sc_meta,
        set_seed=True,
        seed=i,
        spatial_cell_type=None,
        spatial_dim=2,
        spatial_size=50,
        delta=10,)
    
    ## Uncomment the following line to save the generated sc_meta files, which are used in Fig2 Panel D
    # generate_sc_meta_new.to_csv(f'{script_dir}/Panel_D_data/sccube/sc_meta_{i}.csv')